In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_dir = r"D:\Final_Project\CNN\chest_xray\train"
val_dir   = r"D:\Final_Project\CNN\chest_xray\val"
test_dir  = r"D:\Final_Project\CNN\chest_xray\test"

In [3]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
)

val_test_gen = ImageDataGenerator(rescale=1./255)
train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

val_data = val_test_gen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

test_data = val_test_gen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False
)

Found 4490 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 190 images belonging to 2 classes.


# ResNet50 Transfer Learning

In [4]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D

base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False  # Freeze base model

94765736/94765736 [==============================] - 9s 0us/step


In [5]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

# Train ResNet50

In [6]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

model.save(r"D:\Final_Project\CNN\resnet50_pneumonia.h5")


Epoch 1/10
141/141 [==============================] - 193s 1s/step - loss: 0.5458 - accuracy: 0.7637 - val_loss: 0.7322 - val_accuracy: 0.5000
Epoch 2/10
141/141 [==============================] - 191s 1s/step - loss: 0.5038 - accuracy: 0.7726 - val_loss: 0.7643 - val_accuracy: 0.5000
Epoch 3/10
141/141 [==============================] - 191s 1s/step - loss: 0.4778 - accuracy: 0.7731 - val_loss: 0.8218 - val_accuracy: 0.5000
Epoch 4/10
141/141 [==============================] - 190s 1s/step - loss: 0.4578 - accuracy: 0.7753 - val_loss: 0.7404 - val_accuracy: 0.5625
Epoch 5/10
141/141 [==============================] - 289s 2s/step - loss: 0.4346 - accuracy: 0.7797 - val_loss: 0.8645 - val_accuracy: 0.5000
Epoch 6/10
141/141 [==============================] - 29079s 208s/step - loss: 0.4133 - accuracy: 0.7886 - val_loss: 0.8143 - val_accuracy: 0.5625
Epoch 7/10
141/141 [==============================] - 492s 3s/step - loss: 0.3921 - accuracy: 0.7893 - val_loss: 0.8398 - val_accuracy: 0.

In [8]:

model.save(r"D:\Final_Project\CNN\resnet50_pneumonia.h5")

In [7]:
from tensorflow.keras.applications import MobileNetV2

base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

model.save(r"D:\Final_Project\CNN\mobilenet_pneumonia.h5")


9406464/9406464 [==============================] - 1s 0us/step
Epoch 1/10
141/141 [==============================] - 121s 838ms/step - loss: 0.2382 - accuracy: 0.9016 - val_loss: 0.3218 - val_accuracy: 0.8125
Epoch 2/10
141/141 [==============================] - 178s 1s/step - loss: 0.1441 - accuracy: 0.9448 - val_loss: 0.4934 - val_accuracy: 0.8125
Epoch 3/10
141/141 [==============================] - 216s 2s/step - loss: 0.1253 - accuracy: 0.9535 - val_loss: 0.4301 - val_accuracy: 0.7500
Epoch 4/10
141/141 [==============================] - 215s 2s/step - loss: 0.1107 - accuracy: 0.9577 - val_loss: 0.7760 - val_accuracy: 0.7500
Epoch 5/10
141/141 [==============================] - 106s 748ms/step - loss: 0.1107 - accuracy: 0.9566 - val_loss: 0.6687 - val_accuracy: 0.7500
Epoch 6/10
141/141 [==============================] - 184s 1s/step - loss: 0.1062 - accuracy: 0.9566 - val_loss: 0.6329 - val_accuracy: 0.7500
Epoch 7/10
141/141 [==============================] - 105s 735ms/step - l

In [9]:
test_loss, test_acc = model.evaluate(test_data)
print("Test Accuracy:", round(test_acc * 100, 2), "%")

6/6 [==============================] - 7s 1s/step - loss: 0.1761 - accuracy: 0.9263
Test Accuracy: 92.63 %


In [10]:
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score
)
# Predict probabilities
y_prob = model.predict(test_data)

# Convert probabilities to class labels
y_pred = (y_prob >= 0.5).astype(int).ravel()

# True labels
y_true = test_data.classes

6/6 [==============================] - 4s 490ms/step


In [11]:
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print("Precision:", round(precision, 4))
print("Recall:", round(recall, 4))
print("F1 Score:", round(f1, 4))

Precision: 0.8958
Recall: 0.9556
F1 Score: 0.9247


In [12]:
class_names = list(test_data.class_indices.keys())

print("\nClassification Report:\n")
print(classification_report(
    y_true,
    y_pred,
    target_names=class_names
))


Classification Report:

              precision    recall  f1-score   support

      NORMAL       0.96      0.90      0.93       100
   PNEUMONIA       0.90      0.96      0.92        90

    accuracy                           0.93       190
   macro avg       0.93      0.93      0.93       190
weighted avg       0.93      0.93      0.93       190



In [13]:
cm = confusion_matrix(y_true, y_pred)

print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[90 10]
 [ 4 86]]


In [17]:
folder = r"D:\Final_Project\CNN\chest_xray\train\NORMAL"
import os
import numpy as np
from tensorflow.keras.preprocessing import image
img_name = os.listdir(folder)[19]
img_path = os.path.join(folder, img_name)
# Load image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

In [18]:
# Predict
prediction = model.predict(img_array)
prob = prediction[0][0]

print("Probability of PNEUMONIA:", round(prob, 4))

if prob >= 0.5:
    print("Prediction: PNEUMONIA")
else:
    print("Prediction: NORMAL")

1/1 [==============================] - 0s 41ms/step
Probability of PNEUMONIA: 0.0191
Prediction: NORMAL
